In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 335kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670922 sha256=807eef4024dd14d34f91f5bee2d6ac197d5027bb897f74b088dc3d9b77514998
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
from surprise import Reader, Dataset, SVD

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# Import libraries
import numpy as np
import pandas as pd
PATH = '/content/drive/My Drive/TCCDataScience/datasets/mlsmall/'
movies = pd.read_csv(PATH+'movies.csv')
#genome_scores = pd.read_csv(PATH+'genome-scores.csv')
tags = pd.read_csv(PATH+'tags.csv')
#genome_tags = pd.read_csv(PATH+'genome-tags.csv')
#Use ratings data to downsample tags data to only movies with ratings 
ratings = pd.read_csv(PATH+'ratings.csv')

In [5]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [6]:
n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 610 | Number of movies = 9724


In [7]:
Ratings = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
R = Ratings.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [10]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of MovieLens1M dataset is 98.3%


In [11]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [12]:
sigma = np.diag(sigma)

In [13]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [14]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,1.593259,-0.474160,0.237581,0.346021,0.053504,-0.108999,1.355739,-0.426912,-0.077438,0.284485,0.417117,0.245054,0.362827,-0.139144,0.255003,0.067873,-0.220421,-0.033781,0.076522,-0.159481,-0.072023,-0.318595,0.774845,1.631756,-0.336022,0.139459,0.739682,-0.014906,0.084650,0.541279,0.134242,...,-0.178088,0.082812,0.039631,0.069620,0.039631,0.033760,0.016202,0.151717,0.036710,-0.172538,-0.155189,-0.121909,0.070459,0.038139,0.052525,0.090319,0.010978,0.012984,-0.018321,0.050599,-0.008879,-0.085795,-0.013370,0.006966,0.016995,0.004960,0.006966,0.004960,0.004960,-0.030425,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,-0.152530,0.077881,0.064805,0.005182,-0.001014,0.045648,0.096822,-0.027822,-0.004426,0.108646,-0.008286,-0.077978,0.016521,0.009482,0.023615,-0.212386,0.040676,-0.001671,-0.021906,-0.174248,0.013497,0.031743,-0.202221,-0.163632,0.030075,0.000392,-0.033238,0.000224,0.017713,0.009105,0.033852,...,0.107281,-0.026524,0.027091,0.011932,0.027091,0.016864,0.051425,0.069516,0.057564,-0.006654,-0.005944,0.014142,-0.002021,0.038409,0.035970,-0.002728,0.002845,0.003324,0.019168,-0.001314,-0.001049,-0.003101,0.009848,0.001888,0.004281,0.001410,0.001888,0.001410,0.001410,0.024940,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,-0.059362,-0.005484,0.031175,0.019045,-0.008570,0.009015,0.029135,-0.021266,0.017135,-0.058321,-0.012562,-0.076699,0.011520,-0.014717,0.041779,-0.040655,-0.004429,-0.009282,-0.009510,0.071715,0.010940,0.023559,0.023674,-0.001701,0.006543,0.006831,0.067326,0.007865,-0.001794,0.011360,0.001821,...,0.010848,-0.005258,0.012808,0.017701,0.012808,0.012132,0.009292,0.002576,0.012102,-0.001268,-0.000341,0.020286,0.014689,0.004859,0.014408,0.016599,0.008085,0.008100,0.003702,0.012780,0.007564,0.003369,0.005910,0.008053,0.008131,0.008038,0.008053,0.008038,0.008038,0.005310,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,-0.017365,0.637032,-0.092990,0.280611,0.268450,-0.027183,0.161275,1.801845,-0.258694,-0.035952,0.131731,1.912042,-0.169062,-0.034940,0.063111,1.661501,0.158679,0.026145,0.551609,0.492758,0.256954,-0.220697,2.094105,1.940568,0.880095,-0.077526,1.327586,0.003094,0.087768,0.102798,0.069933,...,-0.288689,0.111226,0.004181,0.059797,0.004181,0.006699,-0.011445,0.190691,-0.072316,0.121225,0.109122,-0.003448,-0.019439,0.049429,0.005512,-0.025047,-0.002677,-0.003250,0.010189,-0.013831,0.002714,0.060707,-0.022289,-0.001531,-0.004396,-0.000958,-0.001531,-0.000958,-0.000958,0.006164,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,1.023455,0.974908,-0.070313,0.135688,0.237220,0.208138,0.552464,0.771266,-0.049298,0.326467,0.082279,1.158006,0.332666,0.156395,0.109681,0.861100,0.242800,0.033883,0.124262,0.044031,-0.000574,0.465793,1.254308,1.893785,0.999793,-0.017239,1.256341,-0.011412,0.111539,-0.012218,0.110956,...,-0.007826,0.023460,-0.006995,-0.012447,-0.006995,-0.004094,-0.009139,0.025778,0.004529,-0.017096,-0.015755,-0.002459,-0.006417,-0.003125,-0.008097,-0.007197,0.003527,0.004970,-0.009438,-0.005636,-0.003344,-0.010391,-

In [ ]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print  ('User {0} has already rated {1} movies.' .format(userID, user_full.shape[0]))
    print ( 'Recommending highest {0} predicted ratings movies not already rated.' .format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds, 1310, movies, ratings, 20)


IndexError: ignored

In [15]:
from surprise import Reader, Dataset, SVD

In [19]:
# Load Reader library
reader = Reader()
from surprise.model_selection import cross_validate

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# We'll use the famous SVD algorithm.
algo = SVD()
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8751  0.8726  0.8679  0.8738  0.8773  0.8733  0.0031  
MAE (testset)     0.6732  0.6704  0.6666  0.6707  0.6751  0.6712  0.0029  
Fit time          5.29    5.42    5.36    5.34    5.34    5.35    0.04    
Test time         0.22    0.21    0.14    0.21    0.22    0.20    0.03    


{'fit_time': (5.289286851882935,
  5.420627593994141,
  5.357020854949951,
  5.34176230430603,
  5.3387627601623535),
 'test_mae': array([0.67323352, 0.67043091, 0.66657563, 0.67070584, 0.67508513]),
 'test_rmse': array([0.87507527, 0.87263291, 0.86790702, 0.87378472, 0.87734109]),
 'test_time': (0.22305583953857422,
  0.2127833366394043,
  0.1422264575958252,
  0.2148573398590088,
  0.21849846839904785)}

In [21]:
trainset = data.build_full_trainset()
algo.train(trainset)
accura

AttributeError: ignored

In [22]:
from surprise import   accuracy
from surprise.model_selection import train_test_split

In [23]:

# train SVD on 75% of known rates
trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)

# check the accuracy using Root Mean Square Error
accuracy.rmse(predictions)

RMSE: 0.8765


0.8765415884435029

In [24]:
algorithm.predict(1310, 1994)

Prediction(uid=1310, iid=1994, r_ui=None, est=3.5031723299702575, details={'was_impossible': False})

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold

# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')

# define a cross-validation iterator
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)